## Integrating rigorous desalination models into network optimization
It is often desired to design a desalination plant at a certain desalination site in the network. However, it is not clear what the optimal design and operation of the plant should be. In this notebook we show how to co-optimize the network operation and desalination design by using the integrated optimization model available in PARETO. 

First we import the necessary utilities to read the network data, the function to build the integrated model with MD (Membrane Distillation) as the desalination model choice (`integrated_model_build`) and the Pyomo package.

In [ ]:
from importlib import resources
from pareto.utilities.get_data import get_data
from pareto.models_extra.CM_module.set_param_list import (
    set_list,
    parameter_list)
from pareto.models_extra.CM_module.cm_utils.data_parser import data_parser
from pareto.models_extra.Integrate_desal.integrated_models.integrated_optimization_md import integrated_model_build
import pyomo.environ as pyo

Importing network data

In [ ]:
with resources.path(
            "pareto.case_studies",
            "integrated_desalination_demo.xlsx",
        ) as fpath:
            [df_sets, df_parameters] = get_data(fpath, set_list, parameter_list)

            network_data = data_parser(df_sets, df_parameters)

Along with network data the integrated model inputs a dictionary with the name of the site at which the desalination unit needs to be built and the technology - MD in this case 

In [ ]:
desalination_dict = {"R01_IN": "MD"}

Now we will build the integrated model using the network data and the desalination dictionary. The model initializes the network and desalination units. It takes several seconds to initialize in this case.

In [ ]:
m = integrated_model_build(network_data, treatment_dict=desalination_dict)

# Solving the integrated model
Now we import IPOPT - a non-linear programming solver from the `SolverFactory` available with Pyomo to solve the integrated model.

In [ ]:
ipopt = pyo.SolverFactory("ipopt")
ipopt.options["mu_init"] = 1e-6
ipopt.options["bound_push"] = 1e-6
ipopt.solve(m, tee = True)

# Results
Now that the model is solved, we can obtain the results of our optimization problem. In this case we show how to obtain the various optimal network costs and optimal desalination CAPEX and OPEX

In [ ]:
print("=================================================================")
print("#########            Costing Variables (USD)        #############")
print("=================================================================")
print("Piping cost")
print(pyo.value(m.m_network.arc_cost))   
print("=================================================================")
print("Disposal cost")
print(pyo.value(m.m_network.disp_cost))
print("=================================================================")
print("Fresh water cost")
print(pyo.value(m.m_network.fresh_cost))
print("=================================================================")
print("Storage Cost")
print(pyo.value(m.m_network.stor_cost))
print("=================================================================")
print("Storage reward")
print(pyo.value(m.m_network.stor_rev))
print("=================================================================")
print("Treatment reward")
print(pyo.value(m.m_network.treat_rev))
print("=================================================================")
print("Treatment cost - OPEX")
print(pyo.value(sum(sum(m.OPEX[s, t]/365*m.m_network.p_dt for t in m.m_network.s_T) for s in m.m_network.desalination_nodes)))
print("Treatment cost - CAPEX")
print(pyo.value(sum(m.CAPEX[s] for s in m.m_network.desalination_nodes)/365*m.m_network.p_dt*len(m.m_network.s_T)))
print("=================================================================")